# CPU vs GPU benchmark

CPU를 사용할 때와 GPU를 사용할 때 모델의 학습 속도를 비교한다.

## 아래 코드는 현재 CPU와  GPU를 확인한다

`device_type`에 GPU가 있는지 볼 것.

In [1]:
import tensorflow as tf

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### ※ 참고

`Device(CPU/GPU)`는 텐서를 할당할때부터 사용된다. 따라서 학습데이터와 모델을 정의하는 과정부터 CPU/GPU별로 따로 실행해야한다.

그래서 **함수**로 만들어놓고 두번 호출하도록 했다.

## 데이터를 로드하고 전처리

In [3]:
def prepare_data():
    (X_train, y_train), (_, _) = tf.keras.datasets.cifar10.load_data()

    X_train = X_train.astype('float32') / 255.  # 0~1 사이 값으로 정규화
    
    return X_train, y_train

## 일반 MLP와 CNN 모델 만들기

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

def build_models():
    MLP = Sequential([
        Flatten(input_shape=(32, 32, 3)),
        Dense(256, activation='relu'),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ], name='MLP')

    CNN = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(10, activation="softmax"),
    ], name='CNN')
    
    return MLP, CNN

## CPU와 GPU에서 각각 훈련해보기

In [5]:
# tf.debugging.set_log_device_placement(True)

In [6]:
with tf.device('/CPU:0'):
    X_train, y_train = prepare_data()
    MLP, CNN = build_models()
    # y = [0] ~ [9] 그러므로, sparse_categorical_crossentropy 사용.
    MLP.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    CNN.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    print("* Starting MLP fit...")
    MLP.fit(X_train, y_train, epochs=3)
    print("* Starting CNN fit...")
    CNN.fit(X_train, y_train, epochs=3)
    print()

Metal device set to: Apple M1


2021-11-20 01:05:06.514104: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-20 01:05:06.514188: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


* Starting MLP fit...
Epoch 1/3
  33/1563 [..............................] - ETA: 2s - loss: 2.3695 - accuracy: 0.1619  

2021-11-20 01:05:07.195475: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-20 01:05:07.195638: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1563/1563 [==============================] - 2s 1ms/step - loss: 1.8940 - accuracy: 0.3122
Epoch 2/3
1563/1563 [==============================] - 2s 2ms/step - loss: 1.7232 - accuracy: 0.3764
Epoch 3/3
1563/1563 [==============================] - 2s 2ms/step - loss: 1.6422 - accuracy: 0.4086
* Starting CNN fit...
Epoch 1/3
1563/1563 [==============================] - 10s 7ms/step - loss: 1.4654 - accuracy: 0.4791
Epoch 2/3
1563/1563 [==============================] - 10s 7ms/step - loss: 1.1325 - accuracy: 0.6081
Epoch 3/3
1563/1563 [==============================] - 10s 7ms/step - loss: 1.0063 - accuracy: 0.6539



In [7]:
if tf.config.list_physical_devices("GPU"):
    X_train, y_train = prepare_data()
    MLP, CNN = build_models()
    # y = [0] ~ [9] 그러므로, sparse_categorical_crossentropy 사용.
    MLP.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    CNN.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    print("* Starting MLP fit...")
    MLP.fit(X_train, y_train, epochs=3)
    print("* Starting CNN fit...")
    CNN.fit(X_train, y_train, epochs=3)
    print()
else:
    print("No GPU. Skip.")
    print()

* Starting MLP fit...
Epoch 1/3
   2/1563 [..............................] - ETA: 2:10 - loss: 3.1394 - accuracy: 0.0781

2021-11-20 01:05:46.405140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 9s 6ms/step - loss: 1.9273 - accuracy: 0.2979
Epoch 2/3
1563/1563 [==============================] - 8s 5ms/step - loss: 1.7352 - accuracy: 0.3743
Epoch 3/3
1563/1563 [==============================] - 8s 5ms/step - loss: 1.6566 - accuracy: 0.4053
* Starting CNN fit...
Epoch 1/3
  13/1563 [..............................] - ETA: 13s - loss: 2.3175 - accuracy: 0.0938

2021-11-20 01:06:12.533215: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 12s 7ms/step - loss: 1.4897 - accuracy: 0.4685
Epoch 2/3
1563/1563 [==============================] - 11s 7ms/step - loss: 1.1428 - accuracy: 0.6039
Epoch 3/3
1563/1563 [==============================] - 11s 7ms/step - loss: 1.0142 - accuracy: 0.6474



# 결과
해당 결과는 MacBook Air (M1)에서 나온 결과이다. 다소 충격적인 결과가 나왔는데, GPU를 쓸때보다 오히려 CPU를 쓸때가 더 빨랐다. 내가 코드를 잘못 짠줄 알고 계속 수정해보고 로그도 활성화해보았는데 이게 맞는 결과였다.

스택오버플로우에 질문을 올리려고하니 이미 답변이 있었다.
https://stackoverflow.com/questions/57115833/why-tensorflow-is-slower-with-gpu-instead-of-cpu

**요약하자면, 단순한 모델에서는 GPU를 사용하면 CPU와 GPU간 소통에 드는 오버헤드가 병렬연산의 이점보다 크다는 것이다.**

텐서플로우는 GPU가 존재하는 경우 우선적으로 이를 사용하는데, 때에 따라 단순한 모델인 경우에는 명시적으로 CPU만 사용하는 것이 더 나을수도 있겠다는 새로운 사실을 발견했다.